# GERIT MATCH

In [ ]:
# Aktuelles Arbeitsverzeichnis ermitteln
import os
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/gerit_matching")

Lade Daten

In [ ]:
import pandas as pd

db_hhu = pd.read_csv("data\hhu_db_raw.csv")
db_hhu = db_hhu.sample(n=20, random_state=42)  # random_state für Reproduzierbarkeit


hhu_gerit = pd.read_excel("data/hhu_gerit.xlsx")

Orgas manuell matchen

In [ ]:
# Dictionary für schnellen Lookup aus hhu_gerot erstellen
mapping_dict = dict(zip(hhu_gerit["Einrichtung"], hhu_gerit["Einrichtung"]))

# Exakte Matches zuordnen, nicht gefundene bleiben NaN
db_hhu["matched_organisation"] = db_hhu["organisation_mehrere"].map(mapping_dict)


### LLM laden



In [ ]:
from dotenv import load_dotenv

# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [53]:
import openai
import pandas as pd
from typing import List
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_core.runnables import chain
from langchain.agents.agent_toolkits import create_retriever_tool

# Funktion zur Erstellung von Embeddings mit dem neuen Modell
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Erstelle InMemoryVectorStore für GERIT-Organisationen
vector_store = InMemoryVectorStore(embeddings)

# Alle Organisationen aus GERIT zu den Vektoren hinzufügen
docs = [Document(page_content=einrichtung) for einrichtung in hhu_gerit["Einrichtung"].tolist()]
vector_store.add_documents(docs)

# Definiere den Retriever als Chain mit Scores
@chain
def retriever(query: str) -> List[Document]:
    results = vector_store.similarity_search_with_score(query, k=5)
    docs, scores = zip(*results) if results else ([], [])
    
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score  # Füge den Score als Metadaten hinzu

    return list(docs)

# Beschreibung für den Retriever
description = (
    "Sucht nach gültigen Eigennamen basierend auf einer ungefähren Eingabe und gibt die Ähnlichkeitsscores zurück. "
    "Falls keine passende Entsprechung gefunden wird, wird 'NA' zurückgegeben."
)

# Retriever-Tool erstellen
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns_with_score",
    description=description,
)


In [58]:
query = "Chemie"
results = retriever.invoke(query)

# Ergebnisse anzeigen
for doc in results:
    print(f"Text: {doc.page_content}, Score: {doc.metadata['score']}")


Text: Fach Chemie, Score: 0.7509780915567487
Text: Fach Physik, Score: 0.5848930936229463
Text: Institut für Physikalische Chemie, Score: 0.5569949476123793
Text: Fach Pharmazie, Score: 0.5564846229056805
Text: Lehrstuhl für Organische Chemie, Score: 0.5424619572449465


In [48]:

# Funktion zum Abgleichen von Organisationen
def match_organisation(query, mapping_dict, retriever):
    if not isinstance(query, str):
        return None  # Falls query kein String ist
    
    orgs = [org.strip() for org in query.split(';')]  # Aufteilen und Leerzeichen entfernen
    matched_orgs = []

    for org in orgs:
        # 1. Prüfen, ob eine exakte Übereinstimmung existiert
        if org in mapping_dict:
            matched_orgs.append(mapping_dict[org])
        else:
            # 2. Falls keine exakte Übereinstimmung, unscharfe Suche über den Retriever
            results = retriever.get_relevant_documents(org)

            # Sicherstellen, dass ein Ergebnis existiert und den Score-Threshold erfüllt
            if results and results[0].metadata.get("score", 0) >= 0.5:
                matched_orgs.append(results[0].page_content)

    return "; ".join(matched_orgs) if matched_orgs else None  # Falls Matches vorhanden, mit "; " verbinden




In [ ]:

# Dictionary für exakte Übereinstimmungen erstellen
mapping_dict = dict(zip(hhu_gerit["Einrichtung"], hhu_gerit["Einrichtung"]))

# # Anwenden auf DataFrame mit exakten + unscharfen Matches
# db_hhu["matched_organisation"] = db_hhu["organisation_mehrere"].apply(
#     lambda x: match_organisation(x, mapping_dict, retriever)
# )





# Zuordnung der Organisationen aus db_hhu zu den GERIT-Organisationen
db_hhu["matched_organisation"] = db_hhu["organisation_mehrere"].apply(
    lambda x: match_organisation(x, mapping_dict, retriever)
)
# Ergebnisse ausgeben
print(db_hhu)


                                                   titel  \
9970   Koloniale Spuren im Bergischen Land (unter bes...   
10389                   Campaigning - analog und digital   
5485    Praktikum zum Kurs der Zahnärztlichen Radiologie   
5848                  Der Kalte Krieg in Ostmitteleuropa   
9671   FÄLLT AUS: Grünes NRW.  Umweltverschmutzung, B...   
20122  DFS4120 Deutschkurs für Studierende naturwisse...   
12026  Communication Skills bei neurologischen Erkran...   
10543             BW31 - Kurs 1 & 2: Innovationsökonomik   
385                      Raumbuchung Berufungskommission   
2312                             Hebräische Sprachpraxis   
10392  Konstruktiver Journalismus: Begriff, Zuwendung...   
16839             MW42 - Kurs 1: Entrepreneurial Finance   
19292                                     English Syntax   
14133  Experimentelle Übungen Bio 110: Zell und Molek...   
14267                       Mitarbeiterseminar AG Nowack   
17217                     Recht der digi